<h1>CURRENTS Credit Management Model V1</h1>


<h2>Model introduction</h2>

The model simulates a single user who makes random trades within a marketplace using a mutual credit currency. Since users can spend into negative balance, the basis of a good mutual credit design is the appropriate management of credit. The model is designed to help test the appropriateness of various credit management logics.   

### Questions

1. Given varying variables in the credit management logic, how is user behaviour effected?
2. Given different user types, what credit management logic best supports the overall aims of the ecosystem design (i.e. does their balance tend towards 'equilibrium').
3. Does the system accomodate different categories of users? For example, both high & low volume transactors? 

### Assumptions

1. Users who maintain equilibrium are generally rewarded with larger credit access. 
2. For some portion of users, transaction volumes will increase over time as they become key nodes within their respective ecosystems. Consequently, their credit allowance should increase with volume provided they maintain sufficient balance equilibrium over time. 

### Constraints / Scope

* The intention of this model is to generate insights that wil inform the design of the 'Currents' credit management algorithm

# 0. Dependencies

In [898]:
# Standard libraries: https://docs.python.org/3/library/
import math
from numpy import random
import numpy as np

# Analysis and plotting modules
import pandas as pd
# import plotly
import plotly.express as px
from random import normalvariate

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionContext, Executor


# 1. State Variables & System Parameters

In [899]:
transactionList = []
balanceHistory = [] # a record of the user's running balance'
avgBalanceHistory = []
sendVolHistory = []
time = 1000



# states are defined as python dictionaries:
initial_state = {
    'userBalance' : 0,
    'avgUserBalance' : 0,
    'totalNumTransactions' : 0,
    'creditLimit' : -500,
    'userCapacity' : 500,
    'sendVolume' : 0, # volume over X send transactions specified by 'sumVolume' param
    'avgSendVolume': 0, # used to smooth credit allowance
    'transaction' : 0
}


# System parameters:

system_params = {
    'integrity_coefficient': [1], # determines user's balance proclivity (1.0 = balanced spending & receiving, 2 = spends more, etc.)
    'AVG_ammount': [100], # Average transaction ammount
    'sumVolume': [30], # How many previous transactions to use in credit calculation (i.e. 'send volume')
    'creditRestriction': [1] # 1 = on, 0 = off (turn off to disable restrictions on spending)
    
}



MONTE_CARLO_RUNS = 10

seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)] #flexible way to create unique seeds for each monte carlo run


# Policy functions

In [900]:
# Generate random incoming and outgoing User transactions
def p_transact(params, substep, state_history, previous_state):
    
    run = previous_state['run']
    integ = params['integrity_coefficient']
    capacity = previous_state['userCapacity']
    avgTransaction = params['AVG_ammount'] #average transaction size 
    
    # randomly switch between sending (- value) or receiving (+)
    posNeg = [-1, 1]
    switch = posNeg[seeds[run-1].randint(0,2)]
    
    # generate random transaction amount
    transaction = avgTransaction * (seeds[run-1].rand()+0.5) 
    
    # randomly switch to sending/receiving 
    transaction = transaction * switch
    
    #add some random large transactions to the mix
    y = seeds[run-1].rand()
    if(y > 0.9):
        transaction *= (5*y)
          
    # Bias outgoing transactions based on 'integrity' param (allows testing of different spending proclivities)
    y = seeds[run-1].rand()
    if(y > 0.9 and transaction < 0):
        transaction *= integ # make some outgoing (determined by user integrity)
    
    if(params['creditRestriction'] == 1):    
        if(transaction < 0 and (transaction*-1) > capacity): # if outgoing transaction, ensure it doesn't exceed current capacity
            transaction = capacity*-1 
        
        
    # update total number of transactions
    x = previous_state['totalNumTransactions'] + 1
    
    return {'transaction_ammount': transaction, 'totalNumTransactions': x}

    
def p_updateUserCapacity(params, substep, state_history, previous_state):
    
    x = previous_state['userBalance'] - previous_state['creditLimit']
    return {'newUserCapacity': x}



# Sum X number of outgoing transactions (-) (i.e. payments) based on 'sumVolume' parameter (informs credit allowance)
def p_calculateSendVolume(params, substep, state_history, previous_state):
    
    y = params['sumVolume']
    sendVol = 0
    payments = [timestep[-1]["transaction"] for timestep in state_history if (timestep[-1]["transaction"]) < 0]
    
    # sum X number of outgoing transactions according to sumVolume
    if(y > len(payments)):
        sendVol = np.sum(payments)
    else:
        for x in payments[(-1*y):]:
            sendVol += x
    
    return {'sendVolume': sendVol*-1}





def p_updateAVGSendVol(params, substep, state_history, previous_state):
    
    mean_sendVol = np.mean([timestep[-1]["sendVolume"] for timestep in state_history])
    
    return {'avgSendVolume': mean_sendVol}



# new updateAVGBalance
def p_updateAVGBalance(params, substep, state_history, previous_state):
    
    mean_balance = np.mean([timestep[-1]["userBalance"] for timestep in state_history])

    return {'avgUserBalance': mean_balance}



# CREDIT MANAGEMENT LOGIC

def p_updateCreditLimit(params, substep, state_history, previous_state):
    
    # fetch appropriate variables
    defaultCreditLimit = -500
    avgBalance = previous_state['avgUserBalance']
    avgSendVol = previous_state['avgSendVolume']
    c_limit = previous_state['creditLimit']
    avgTrans = params['AVG_ammount']
    numTransactions = previous_state['totalNumTransactions']
    
    
    # CREDIT LOGIC
    
    # Before we have enough transactional data     
    if(numTransactions < 10):
        c_limit = previous_state['creditLimit']
        
    else:
        
        # CALCULATE CHANGE ACCORDING TO ANGLE OF SLOPE (avgBalance)
        sensitivityA = 1
        delta = 1 # average slope over Time = delta (increase for more general trendline)

        avgBalanceList = [timestep[-1]["avgUserBalance"] for timestep in state_history]

        y1 = avgBalanceList[(-1*(delta+1)):][0]
        y2 = avgBalanceList[-1:][0]

        delta_y = y2 - y1
        slope = delta_y / delta
        q = 0

        # modulate credit allowance according to the angle of slope
        if(avgBalance>0): 
            q = slope*sensitivityA # lower credit limit (i.e. increase spending capacity)
        else: 
            q = -1*slope*sensitivityA # raise credit lmit (i.e. decrease spending capacity
        
          
        # CALCULATE CHANGE ACCORDING TO EQUILIBRIUM (i.e. RATIO = AvgBalance / SendVolume)
        y = 0
        if(avgSendVol != 0): #ensure no division by zero
            if(avgBalance<0): #change negative balance to positive, if necessary
                avgBalance *= -1
            x = avgBalance/avgSendVol 
            y = -4*x + 1  #linear function (when y = 0, x = 1/4)           
            y = y*-1
            
        rate = 30 # sensitivity
        c_limit = (c_limit + (y*rate) + (q*avgSendVol)/800) 
    
    
    # ensure credit limit doesn't go above zero
    if(c_limit > defaultCreditLimit):
        c_limit = defaultCreditLimit  
    if(c_limit < (avgSendVol*-1)): # limit credit allowance to average send volume (over x transactions, specified as "sumVolume" param)
        c_limit = avgSendVol*-1
    

    return {'creditLimit': c_limit}
    
    

# State Update Functions

In [901]:
# Update current balance based previous transaction
def s_userBalance(params, substep, state_history, previous_state, policy_input):
    
    newBalance = previous_state['userBalance'] + policy_input['transaction_ammount'] 
    x = (int(round(newBalance)))
    return ('userBalance', x)

# update current average balance (note: distinct from current 'userBalance')
def s_AVGBalance(params, substep, state_history, previous_state, policy_input):
    x = policy_input['avgUserBalance']
    
    return ('avgUserBalance', x)


def s_totalNumTransactions(params, substep, state_history, previous_state, policy_input):
    x = policy_input['totalNumTransactions']
    return ('totalNumTransactions', x)


def s_userCapacity(params, substep, state_history, previous_state, policy_input):
    x = policy_input['newUserCapacity']
    y = int(round(x))
    return ('userCapacity', y)


def s_userSendVolume(params, substep, state_history, previous_state, policy_input):  
    x = policy_input['sendVolume']
    
    return ('sendVolume', x)


def s_avgSendVolume(params, substep, state_history, previous_state, policy_input):
    x = policy_input['avgSendVolume']
    return ('avgSendVolume', x)
    

def s_creditLimit(params, substep, state_history, previous_state, policy_input):
    x =  policy_input['creditLimit']
    y = int(round(x))
    return ('creditLimit', y)


def s_transaction(params, substep, state_history, previous_state, policy_input):
    
    x = policy_input['transaction_ammount']
    y = int(round(x))
    
    return ('transaction', y)



In [902]:
partial_state_update_blocks = [
    {
        'policies': {
            'transact': p_transact,
            'updateUserCapacity': p_updateUserCapacity,
            'calculateSendVolume': p_calculateSendVolume,
            'avgSendVolume': p_updateAVGSendVol,
            'avgUserBalance' : p_updateAVGBalance,
            'updateCreditLimit': p_updateCreditLimit
        },
        'variables': {
            'userBalance': s_userBalance,
            'avgUserBalance': s_AVGBalance,
            'creditLimit': s_creditLimit,
            'userCapacity': s_userCapacity,
            'sendVolume': s_userSendVolume,
            'avgSendVolume': s_avgSendVolume,
            'transaction': s_transaction,
            'totalNumTransactions': s_totalNumTransactions
                      
        }
    }
]



In [903]:
from cadCAD import configs
del configs[:] # Clear any prior configs

#reset global variables
transactionList.clear()
balanceHistory.clear()
avgBalanceHistory.clear()
sendVolHistory.clear()

sim_config = config_sim({
    'N': 10,
    'T': range(time),
    'M': system_params
})


experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)



In [904]:
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1000, 4, 10, 8)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SubsetIDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Ns       : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
ExpIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 4.26s


In [905]:
simulation_result = pd.DataFrame(system_events)
simulation_result.head(143)

,userBalance,avgUserBalance,totalNumTransactions,creditLimit,userCapacity,sendVolume,avgSendVolume,transaction,simulation,subset,run,substep,timestep
0,0,0.000000,0,-500,500,0.0,0.000000,0,0,0,1,0,0
1,-109,0.000000,1,0,500,-0.0,0.000000,-109,0,0,1,1,1
2,-17,-54.500000,2,0,-109,109.0,0.000000,92,0,0,1,1,2
3,92,-42.000000,3,0,-17,109.0,36.333333,109,0,0,1,1,3
4,195,-8.500000,4,-36,92,109.0,54.500000,103,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,-1838,342.710145,138,-2370,514,3792.0,3082.500000,-57,0,0,1,1,138
139,-1718,327.021583,139,-2446,532,3756.0,3087.604317,120,0,0,1,1,139
140,-1043,312.414286,140,-2524,728,3756.0,3092.378571,675,0,0,1,1,140
141,-1108,302.801418,141,-2598,1481,3756.0,3097.085106,-65,0,0,1,1,141


### 8. Simulation Output Preparation

In [906]:
# Get system events and attribute index
df = (pd.DataFrame(system_events)
        .assign(days=lambda df: df.timestep)
        .query('timestep > 2')
        .query('timestep < 1000')
     )

# Clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
    

In [907]:
pd.options.plotting.backend = "plotly"

# plot something else by querying the dataframe
fig_df = df.query('run == 4')
# fig_df = df

fig_df.plot(
    kind='line',
    x='timestep',
    y=['avgUserBalance', 'creditLimit', 'transaction', 'sendVolume']
)

In [908]:
fig = px.line(
    df,
    x='timestep',
    y=['avgUserBalance', 'creditLimit', 'sendVolume'],
    facet_row='simulation',
    facet_col='run',
    height=400,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=30, r=20, t=20, b=20),
)

fig.show()

In [909]:
# fig_df = df.query()
fig_df = df

fig = px.scatter(
    fig_df,
    x=fig_df.days,
    y=fig_df.userBalance,
    color=fig_df.integrity_coefficient.astype(str),
    opacity=0.1,
    trendline="lowess",
    labels={'color': 'integrity_coefficient'}
)

fig.show()

In [910]:
fig3 = px.scatter(
    df[df.timestep >= 0],
    x='avgUserBalance',
    y='creditLimit',
    color='timestep'
)
fig3.show()